In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os
import time

In [15]:
# los links siguen el formato: https://www.pagina12.com.ar/secciones/economia?page=166
# por lo que podemos jugar con el numero para ir recorriendo las noticias de un mes determinado

pagina_inicio = 124 # primera noticia del mes
pagina_final = 101 # ultima noticia del mes

In [16]:
# example
# page_url =  'https://www.clarin.com/economia/page/4'
page_url =  'https://www.pagina12.com.ar/secciones/economia?page=166'

In [17]:
news_url = 'https://www.pagina12.com.ar/800887-el-gobierno-bonaerense-multo-a-flybondi-por-300-millones'
# news_url = 'https://www.clarin.com/economia/dolar-blue-hoy-cotiza-sabado-01-febrero_0_674SqKuAFv.html'

In [18]:
# Funciones

# obtiene los titulos y urls de la pagina principal
def get_news_list(page_url): # e.g. 'https://www.pagina12.com.ar/secciones/economia?page=166'
    response = requests.get(page_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser') # parsear el contenido de la pagina
    noticias = soup.find_all('article') # encuentra los elementos <article>
    
    news_data = []
    
    for noticia in noticias: # recorremos la lista de articulos
        anchor = noticia.find('a')  # <a> element containing both title and link
        if anchor:
            # title = title_tag.get_text(strip=True)
            title = anchor.get('aria-label')
            if title:
                prefix = "Ir a la nota "
                if title.startswith(prefix):
                    title = title[len(prefix):]
            
            link = anchor['href']
            if not link.startswith('http'):
                link = 'https://www.pagina12.com.ar' + link
            
            news_data.append({'title': title, 'link': link})
            
    return news_data

# recorrer lista de links y obtener contenido y fecha de la noticia
def get_news_content_and_date(news_url):
    response = requests.get(news_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extraer fecha desde <time class="createDate" datetime="...">
    fecha = None
    time_tag = soup.find('time', class_='createDate')
    if time_tag and time_tag.has_attr('datetime'):
        fecha = time_tag['datetime']
    else:
        # Backup: buscar otras formas por si cambia el HTML
        time_tag = soup.find('time')
        if time_tag and time_tag.has_attr('datetime'): # esta es la que funciona para pagina 12
            fecha = time_tag['datetime']
        elif time_tag:
            fecha = time_tag.get_text(strip=True)

    # Extraer título
    # titulo_tag = soup.find('h1', class_='storyTitle')
    titulo_tag = soup.find('h1')
    titulo = titulo_tag.get_text(strip=True) if titulo_tag else 'Título no encontrado'

    # Extraer contenido
    content_div = soup.find('div', {'class': ['body', 'article-body']})
    if not content_div:
        content_div = soup.find('article')

    if content_div:
        paragraphs = content_div.find_all('p')
        contenido = '\n'.join(p.get_text(strip=True) for p in paragraphs)
    else:
        contenido = "Contenido no encontrado"
    
    return fecha, titulo, contenido

# funcion final para screappear
def scrape_pages(start_page=pagina_inicio, end_page=pagina_final):
    diario = 'Pagina 12'
    output_dir = './outputs'
    os.makedirs(output_dir, exist_ok=True)
    
    for page_num in range(start_page, end_page - 1, -1):  # start_page > end_page, va descendiendo
        # url = f'https://www.clarin.com/economia/page/{page_num}'
        url = f'https://www.pagina12.com.ar/secciones/economia?page={page_num}'
        print(f'Procesando página {page_num} ...')
        records = []
        
        try:
            news_list = get_news_list(url) # obtener lista de titulos y urls
        except Exception as e:
            print(f"❌ Error al obtener lista de noticias en página {page_num}: {e}")
            continue
        
        for news in news_list:
            try:
                link = news['link']
                fecha, titulo_real, contenido = get_news_content_and_date(link)
                
                # Formatear fecha
                if fecha:
                    try:
                        fecha_dt = datetime.fromisoformat(fecha.replace('Z', '+00:00'))
                        fecha_str = fecha_dt.strftime('%Y-%m-%d')
                    except:
                        fecha_str = fecha
                else:
                    fecha_str = 'Fecha no encontrada'

                records.append({
                    'diario': diario,
                    'fecha': fecha_str,
                    'titulo noticia': titulo_real,
                    'contenido': contenido,
                    'link': link
                })
            except Exception as e:
                print(f"⚠️  Error al procesar noticia {news.get('link', '')}: {e}")
            
            time.sleep(5)  # Espera entre noticias
        
        # Guardar CSV
        df = pd.DataFrame(records)
        output_path = os.path.join(output_dir, f'pagina12_pag_{page_num}_abr.csv')
        df.to_csv(output_path, index=False)
        print(f'✅ Página {page_num} guardada en {output_path}')
        
        time.sleep(5)  # Espera entre páginas

In [19]:
# Ejecuto la función para scrappear la página de Clarín.
if __name__ == "__main__":
    scrape_pages()
print("Terminó.")

Procesando página 124 ...
✅ Página 124 guardada en ./outputs\pagina12_pag_124_abr.csv
Procesando página 123 ...
✅ Página 123 guardada en ./outputs\pagina12_pag_123_abr.csv
Procesando página 122 ...
✅ Página 122 guardada en ./outputs\pagina12_pag_122_abr.csv
Procesando página 121 ...
✅ Página 121 guardada en ./outputs\pagina12_pag_121_abr.csv
Procesando página 120 ...
✅ Página 120 guardada en ./outputs\pagina12_pag_120_abr.csv
Procesando página 119 ...
✅ Página 119 guardada en ./outputs\pagina12_pag_119_abr.csv
Procesando página 118 ...
✅ Página 118 guardada en ./outputs\pagina12_pag_118_abr.csv
Procesando página 117 ...
✅ Página 117 guardada en ./outputs\pagina12_pag_117_abr.csv
Procesando página 116 ...
✅ Página 116 guardada en ./outputs\pagina12_pag_116_abr.csv
Procesando página 115 ...
✅ Página 115 guardada en ./outputs\pagina12_pag_115_abr.csv
Procesando página 114 ...
✅ Página 114 guardada en ./outputs\pagina12_pag_114_abr.csv
Procesando página 113 ...
✅ Página 113 guardada en ./o